# Exercícios 2
- Testar função de correção de *dummies*
- Testar módulo de salvar a imagem

### Importação de módulos

In [1]:
import ee
import pandas as pd
import numpy as np
import geemap
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),os.pardir)))
from src import eetools as eet
from src import saveModule

In [2]:
try: 
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

### Coleta de dados

In [3]:
coordenadas = "-48.53801472648439,-22.503806214013736,-48.270222978437516,-22.7281869567509"
x1, y1, x2, y2 = coordenadas.split(",")

datas = "2014-08-01,2014-12-01"
inicio, fim = datas.split(",")
escala = 30

geom = ee.Geometry.Polygon([[[float(x1),float(y2)],
                             [float(x2),float(y2)],
                             [float(x2),float(y1)],
                             [float(x1),float(y1)],
                             [float(x1),float(y2)]]])

latitude_central = (float(x1)+float(x2))/2
longitude_central = (float(y1)+float(y2))/2

collec = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(geom)\
    .filterDate(inicio,fim)\
    .filterMetadata('CLOUD_COVER','less_than',30)\
    .map(eet.ndvi)

### Extração das séries temporais

In [4]:
ndvi_collec = collec\
    .select('ndvi')\
    .toBands()

ndvi_collec_bands = ndvi_collec\
    .bandNames()\
    .getInfo()

blended = ee.Image(99999).blend(ndvi_collec)

In [5]:
time_series = eet.extrair_lonlat(blended,geom,ndvi_collec_bands,escala)
display(time_series)

,LC08_220076_20140911_ndvi,LC08_220076_20141013_ndvi,LC08_220076_20141029_ndvi,LC08_221075_20140801_ndvi,LC08_221075_20140817_ndvi,LC08_221075_20140902_ndvi,LC08_221075_20140918_ndvi,LC08_221075_20141004_ndvi,LC08_221075_20141105_ndvi,LC08_221076_20140801_ndvi,LC08_221076_20140902_ndvi,LC08_221076_20140918_ndvi,LC08_221076_20141004_ndvi,latitude,longitude
0,0.328715,0.482913,0.465691,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.728169,-48.537927
1,0.705186,0.687115,0.750785,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.728169,-48.537627
2,0.747272,0.724499,0.777710,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.728169,-48.537327
3,0.709579,0.729393,0.819333,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.728169,-48.537027
4,0.737570,0.742866,0.824793,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.728169,-48.536727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667211,0.455296,0.238938,0.299978,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.503906,-48.271634
667212,0.447883,0.242439,0.317909,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.503906,-48.271334
667213,0.458235,0.240911,0.320276,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.503906,-48.271033
667214,0.430154,0.241740,0.319300,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,99999.0,-22.503906,-48.270733


### Correção dos *dummies*

In [6]:
eet.correct_dummies(time_series, 99999.0)

,LC08_220076_20140911_ndvi,LC08_220076_20141013_ndvi,LC08_220076_20141029_ndvi,LC08_221075_20140801_ndvi,LC08_221075_20140817_ndvi,LC08_221075_20140902_ndvi,LC08_221075_20140918_ndvi,LC08_221075_20141004_ndvi,LC08_221075_20141105_ndvi,LC08_221076_20140801_ndvi,LC08_221076_20140902_ndvi,LC08_221076_20140918_ndvi,LC08_221076_20141004_ndvi,latitude,longitude
0,0.328715,0.482913,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,0.465691,-22.728169,-48.537927
1,0.705186,0.687115,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,0.750785,-22.728169,-48.537627
2,0.747272,0.724499,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,0.777710,-22.728169,-48.537327
3,0.709579,0.729393,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,0.819333,-22.728169,-48.537027
4,0.737570,0.742866,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,0.824793,-22.728169,-48.536727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667211,0.455296,0.238938,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,0.299978,-22.503906,-48.271634
667212,0.447883,0.242439,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,0.317909,-22.503906,-48.271334
667213,0.458235,0.240911,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,0.320276,-22.503906,-48.271033
667214,0.430154,0.241740,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,0.319300,-22.503906,-48.270733
